In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import matplotlib.pyplot as plt
import numpy as pd
import sys
from tqdm.notebook import tqdm

In [ ]:
#PyTorch Dataset

In [2]:
class CardClassifierDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform = transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    @property
    def classes(self):
        return self.data.classes
        

In [8]:
#Dataset and Dataloader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_folder = '/aul/homes/amaha038/DeepLearning/Datasets/Card_Dataset_Kaggle/train/'
test_folder = '/aul/homes/amaha038/DeepLearning/Datasets/Card_Dataset_Kaggle/test/'
valid_folder = '/aul/homes/amaha038/DeepLearning/Datasets/Card_Dataset_Kaggle/valid/'


train_dataset = CardClassifierDataset(data_dir = train_folder, transform = transform)
test_dataset = CardClassifierDataset(data_dir = test_folder, transform = transform)
valid_dataset = CardClassifierDataset(data_dir = valid_folder, transform = transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [14]:
#printing the number of features_out for the given model
model_name = 'convnext_base'
base_model = timm.create_model(model_name, pretrained=True)

# Removing the classification head 
features = nn.Sequential(*list(base_model.children())[:-1])

dummy_input = torch.randn(1, 3, 224, 224)

# Passing through the feature extractor
with torch.no_grad():
    output = features(dummy_input)
    print(f"Feature shape: {output.shape}")
    print(f"Flattened feature size: {output.reshape(1, -1).shape[1]}")


Feature shape: torch.Size([1, 1024, 7, 7])
Flattened feature size: 50176


In [15]:
#Simple Model

class SimpleCardClassifier(nn.Module):
    def __init__(self, num_classes=53):
        super(SimpleCardClassifier, self).__init__()
        # Where we define all the parts of the model
        self.base_model = timm.create_model('convnext_base', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        feature_out = 50176
        # Make a classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(feature_out, num_classes)
        )
    
    def forward(self, x):
        # Connect these parts and return the output
        x = self.features(x)
        output = self.classifier(x)
        return output

In [16]:
model = SimpleCardClassifier(num_classes=53)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)


SimpleCardClassifier(
  (base_model): ConvNeXt(
    (stem): Sequential(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
    )
    (stages): Sequential(
      (0): ConvNeXtStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): ConvNeXtBlock(
            (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
            (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=128, out_features=512, bias=True)
              (act): GELU()
              (drop1): Dropout(p=0.0, inplace=False)
              (norm): Identity()
              (fc2): Linear(in_features=512, out_features=128, bias=True)
              (drop2): Dropout(p=0.0, inplace=False)
            )
            (shortcut): Identity()
            (drop_path): Identity()
          )
          (1): ConvNeXtBlock(
      

In [17]:
#loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [18]:
#training loop
num_epochs = 2
train_losses, val_losses = [], []

for epoch in range(num_epochs):
    #Training_Phase
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc='Training loop'):
        #moving inputs and labels to the device
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0) #batch loss across all batches
    train_loss = running_loss / len(train_loader.dataset) # average loss per sample across the whole epoch
    train_losses.append(train_loss)


    #validation_phase
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(valid_loader, desc='Validation loop'):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)

    val_loss = running_loss / len(valid_loader.dataset)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {val_loss}")
    
    
    




Training loop:   0%|          | 0/239 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/9 [00:00<?, ?it/s]

Epoch 1/2 - Train loss: 10.88310015739449, Validation loss: 4.775770446489442


Training loop:   0%|          | 0/239 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/9 [00:00<?, ?it/s]

Epoch 2/2 - Train loss: 4.5749626264742265, Validation loss: 4.647456007183723
